In [40]:
PROJECT1: WIFE-CARRYING
Name: Veera Tegelberg
email: veera.tegelberg@gmail.com

In this project, I am about to look at statistics of Wife-carrying World Championships.
Wife-carrying is a sport invented in Finland back in 1990s. Every July in Sonkajärvi, there are dozens of couples from all around the world attending the competition.
If you are interested in the weird concept, please check this video: https://www.youtube.com/watch?v=p7IM9f16QZ4. (Yes, they have copied the sport to the States too: https://www.youtube.com/watch?v=aCYC9eT6UVQ, but the official championships are organised in Finland.)

In this notebook, I scrape from Wikipedia article a list of "World champions". It includes all winning couples since the beginnign of competition in 1997. 
The list is messy, so I can practice cleaning along with scraping.
https://en.wikipedia.org/wiki/Wife-carrying

PART 1: SCRAPING AND GETTING A DATAFRAME

In [40]:
#First I bring the needed libraries.

#Pandas for cleaning and organizing my data.
import pandas as pd

#Import requests for scraping.
import requests

#Import bs4 for parsing the scraped data.
from bs4 import BeautifulSoup

In [41]:
#To get the html, I make a variable, get it with request and turn it into DOM (text).
url = "https://en.wikipedia.org/wiki/Wife-carrying"
html = requests.get(url).text

In [42]:
#I turn it into on BeautifulSoup object to handle it here in the notebook.
soup = BeautifulSoup(html)

#I print it to see, does it look about right.
soup

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-enabled vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Wife-carrying - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-l

In [49]:
#Let's select the html list, that contains world champions. It is an unordered list (ul).
#There are several uls on the Wikipedia webpage. I poke around to find the correct list (number 30).
#I make variable called champions.
champions = soup.select("ul")[30]

#I print it out to see, if it looks about right.
print(champions)

#It looks great!

<ul><li>1997 – <a class="new" href="/w/index.php?title=Jouni_Jussila&amp;action=edit&amp;redlink=1" title="Jouni Jussila (page does not exist)">Jouni Jussila</a> (Finland) and <a class="new" href="/w/index.php?title=Tiina_Jussila&amp;action=edit&amp;redlink=1" title="Tiina Jussila (page does not exist)">Tiina Jussila</a> (Finland), 65.0 seconds.<sup class="reference" id="cite_ref-8"><a href="#cite_note-8">[8]</a></sup></li>
<li>1998 – <a class="new" href="/w/index.php?title=Imre_Ambos&amp;action=edit&amp;redlink=1" title="Imre Ambos (page does not exist)">Imre Ambos</a> (Estonia) and <a class="new" href="/w/index.php?title=Annela_Ojaste&amp;action=edit&amp;redlink=1" title="Annela Ojaste (page does not exist)">Annela Ojaste</a> (Estonia), 69.2 seconds.<sup class="reference" id="cite_ref-9"><a href="#cite_note-9">[9]</a></sup></li>
<li>1999 – Imre Ambos (Estonia) and Annela Ojaste (Estonia), 64.5 seconds.</li>
<li>2000 – Margo Uusorg (Estonia) and Birgit Ulrich (Estonia),<sup class="ref

In [50]:
#Now I want to get elements for my dataframe.

#Let's make a new list variable of the scraped data, called 'data'.
data = []

#A for loop goes through each element in the previously created champions variable.
for champion in champions:
    #Here I turn everything from variable 'champion' to text.
    text = champion.get_text()
    # I split the text first into two parts based on hyphen in the html.
    parts = text.split(' – ')
    #This means that if there becomes to parts after splitting each row with hyphen, then proseed.
    if len(parts) == 2:
        #Here I give names ('year' and 'champion_info') for two different parts (df columns in the end).
        year, champion_info = parts
        # I want more specif columns in my df, so I split champion_info into 'competitors' and 'time' using comma (,).
        champion_info_parts = champion_info.split(',')
        if len(champion_info_parts) == 2:
            competitors, time = champion_info_parts
            # Further I split competitors into two parts using 'and'. I want names of the carrier and carried in their own columns.
            competitor_parts = competitors.split(' and ')
            if len(competitor_parts) == 2:
                carrier, carried = competitor_parts

            #With append, I create a list of dictionaries from variables I just created above.
            #.strip() takes extra white spaces away.
            data.append({
            'Year': year.strip(),
            'Carrier': carrier.strip(),
            'Carried': carried.strip(),
            'Time': time.strip()
            })


# I create a DataFrame from the extracted data.
df = pd.DataFrame(data)

df.head()

,Year,Carrier,Carried,Time
0,1997,Jouni Jussila (Finland),Tiina Jussila (Finland),65.0 seconds.[8]
1,1998,Imre Ambos (Estonia),Annela Ojaste (Estonia),69.2 seconds.[9]
2,1999,Imre Ambos (Estonia),Annela Ojaste (Estonia),64.5 seconds.
3,2000,Margo Uusorg (Estonia),Birgit Ulrich (Estonia),[10] 55.5 seconds.
4,2001,Margo Uusorg (Estonia),Birgit Ulrich (Estonia),55.6 seconds.[11]


In [51]:
#Just to show and check, this is how the data looks like now.
data

[{'Year': '1997',
  'Carrier': 'Jouni Jussila (Finland)',
  'Carried': 'Tiina Jussila (Finland)',
  'Time': '65.0 seconds.[8]'},
 {'Year': '1998',
  'Carrier': 'Imre Ambos (Estonia)',
  'Carried': 'Annela Ojaste (Estonia)',
  'Time': '69.2 seconds.[9]'},
 {'Year': '1999',
  'Carrier': 'Imre Ambos (Estonia)',
  'Carried': 'Annela Ojaste (Estonia)',
  'Time': '64.5 seconds.'},
 {'Year': '2000',
  'Carrier': 'Margo Uusorg (Estonia)',
  'Carried': 'Birgit Ulrich (Estonia)',
  'Time': '[10] 55.5 seconds.'},
 {'Year': '2001',
  'Carrier': 'Margo Uusorg (Estonia)',
  'Carried': 'Birgit Ulrich (Estonia)',
  'Time': '55.6 seconds.[11]'},
 {'Year': '2002',
  'Carrier': 'Meelis Tammre (Estonia)',
  'Carried': 'Anne Zillberberg (Estonia)',
  'Time': '63.8 seconds.'},
 {'Year': '2003',
  'Carrier': 'Margo Uusorg (Estonia)',
  'Carried': 'Egle Soll (Estonia)',
  'Time': '60.7 seconds.[12]'},
 {'Year': '2004',
  'Carrier': 'Madis Uusorg (Estonia)',
  'Carried': 'Inga Klauso (Estonia)',
  'Time': '65.

In [52]:
#Now I have four columns in a df.

#I want  one column more. It should be called "Country".
#I have checked, that both 'Carrier' and 'Carried' have always represented the same country.

#To split country information into its own column, I need regular experession library (re).
#I import it.
import re

#This is the data we are operating.
df = pd.DataFrame(data)

# I use regular expression to extract county by defining function first.
def extract_country(name):
    # I make a variable called 'match'. It checks everything that is between parenthesis.
    match = re.search(r'\((.*?)\)', name)
    # This way I get things that are only within the parenthesis, not parenthesis themselves.
    return match.group(1)

# I extract country information and add it to a new .
df['Country'] = df['Carrier'].apply(extract_country)

df.head()


,Year,Carrier,Carried,Time,Country
0,1997,Jouni Jussila (Finland),Tiina Jussila (Finland),65.0 seconds.[8],Finland
1,1998,Imre Ambos (Estonia),Annela Ojaste (Estonia),69.2 seconds.[9],Estonia
2,1999,Imre Ambos (Estonia),Annela Ojaste (Estonia),64.5 seconds.,Estonia
3,2000,Margo Uusorg (Estonia),Birgit Ulrich (Estonia),[10] 55.5 seconds.,Estonia
4,2001,Margo Uusorg (Estonia),Birgit Ulrich (Estonia),55.6 seconds.[11],Estonia


In [53]:
#I clean country information away from 'Carrier' and 'Carried'.

#I use regular expressions again to replace parenthesis with nothing ('').
df['Carrier'] = df['Carrier'].str.replace(r' \((.*?)\)', '', regex=True)
df['Carried'] = df['Carried'].str.replace(r' \((.*?)\)', '', regex=True)
df.head()

,Year,Carrier,Carried,Time,Country
0,1997,Jouni Jussila,Tiina Jussila,65.0 seconds.[8],Finland
1,1998,Imre Ambos,Annela Ojaste,69.2 seconds.[9],Estonia
2,1999,Imre Ambos,Annela Ojaste,64.5 seconds.,Estonia
3,2000,Margo Uusorg,Birgit Ulrich,[10] 55.5 seconds.,Estonia
4,2001,Margo Uusorg,Birgit Ulrich,55.6 seconds.[11],Estonia


In [54]:
#I want to take away from 'Time' column the brackets indicating that there was a link and also text "seconds.".

#With regular expressions I delete brackets [] and anything inside them.
df['Time'] = df['Time'].str.replace(r'\[.*?\]', '', regex=True)

#Here I do almost the same, just replacing '\[.*?\]' with '\s*seconds\.'.
#I put in the end str.strip() to be sure, there won't be any white spaces.
df['Time'] = df['Time'].str.replace(r'\s*seconds\.', '', regex=True).str.strip()

df


,Year,Carrier,Carried,Time,Country
0,1997,Jouni Jussila,Tiina Jussila,65.0,Finland
1,1998,Imre Ambos,Annela Ojaste,69.2,Estonia
2,1999,Imre Ambos,Annela Ojaste,64.5,Estonia
3,2000,Margo Uusorg,Birgit Ulrich,55.5,Estonia
4,2001,Margo Uusorg,Birgit Ulrich,55.6,Estonia
5,2002,Meelis Tammre,Anne Zillberberg,63.8,Estonia
6,2003,Margo Uusorg,Egle Soll,60.7,Estonia
7,2004,Madis Uusorg,Inga Klauso,65.3,Estonia
8,2005,Margo Uusorg,Egle Soll,59.1,Estonia
9,2006,Margo Uusorg,Sandra Kullas,56.9,Estonia


In [55]:
#I check the datatypes.
df.dtypes

Year       object
Carrier    object
Carried    object
Time       object
Country    object
dtype: object

In [56]:
#I want to change 'Time' into float, so I can count or compare winning times.
df['Time'] = df.Time.astype(float)

df_cleaned = df.copy()

# df.dtypes
df_cleaned.dtypes

Year        object
Carrier     object
Carried     object
Time       float64
Country     object
dtype: object

PART 2: ASKING QUESTIONS

This is the fun part. Below I have 6 questions for my df. I use the findings in my text later.

About the dataset it is important to notice, that years 2020 and 2021 are missing. This was due to Covid. Event was not held:
https://en.wikipedia.org/wiki/Wife-carrying 

In [57]:
df_cleaned.head()

,Year,Carrier,Carried,Time,Country
0,1997,Jouni Jussila,Tiina Jussila,65.0,Finland
1,1998,Imre Ambos,Annela Ojaste,69.2,Estonia
2,1999,Imre Ambos,Annela Ojaste,64.5,Estonia
3,2000,Margo Uusorg,Birgit Ulrich,55.5,Estonia
4,2001,Margo Uusorg,Birgit Ulrich,55.6,Estonia


1) How many times Finland has won since 1997?

In [61]:
fin_wins = df_cleaned[df_cleaned['Country'] == 'Finland']

#To print it out, I use f string here for a change. 
print(f'Finland has won {len(fin_wins)} times since 1997.')

Finland has won 11 times since 1997.


2) Show me, which countries have won and how many times?

In [60]:
wins_per_country = df_cleaned.Country.value_counts()
wins_per_country

Country
Finland      11
Estonia      11
Lithuania     2
Russia        1
Name: count, dtype: int64

Wow! So, Finland and Estonia are even.

3) Now, which pairs (carrier + carried) have won championships more than ones? 
(Same carrier may have won with different carried.)

In [86]:
most_wins_per_pair = df_cleaned.groupby(['Carrier', 'Carried', 'Country']).size().reset_index(name='Count')


# most_wins_per_pair = df.groupby(['Carrier', 'Carried']).size().reset_index(name='Count')

most_wins_per_pair = most_wins_per_pair.sort_values(by='Count', ascending=False)

#I show in the table only pairs with more than one win.
# most_wins_per_pair = most_wins_per_pair[most_wins_per_pair['Count'] > 1]

most_wins_per_pair

,Carrier,Carried,Country,Count
10,Taisto Miettinen,Kristiina Haapanen,Finland,6
2,Imre Ambos,Annela Ojaste,Estonia,2
4,Madis Uusorg,Inga Klauso,Estonia,2
5,Margo Uusorg,Birgit Ulrich,Estonia,2
6,Margo Uusorg,Egle Soll,Estonia,2
9,Taisto Miettinen,Katja Kovanen,Finland,2
13,Vytautas Kirkliauskas,Neringa Kirkliauskiene,Lithuania,2
0,Alar Voogla,Kirsti Viltrop,Estonia,1
1,Dmitry Sagal,Anastasia Loginova,Russia,1
3,Jouni Jussila,Tiina Jussila,Finland,1


So, guy called Taisto Miettinen has won with his pair Kristiina Haapanen Wife-carrying championships six times - and twice also with Katja Kovanen. Keep pushing Taisto!

4) How many times and on which years Taisto Miettinen has won the championship?

In [75]:
taisto_wins = df_cleaned[df_cleaned['Carrier'] == 'Taisto Miettinen']  

print(f'Taito has won {len(taisto_wins)} times.')

Taito has won 8.


In [77]:
#With '.tolist' I convert Taito's winning years into a list.
winning_years = taisto_wins['Year'].tolist()

#I change all 'Years' into string before joining them.
print(f'Taisto has won in the years: {", ".join(map(str, winning_years))}')


Taisto has won in the years: 2009, 2010, 2011, 2012, 2013, 2017, 2022, 2023


5) How many times championships have been organised before 2024?

In [73]:
#.nunique tells how many unique values there are in pd df column.
championships_organised = df_cleaned['Year'].nunique()

championships_organised

25

6) What is the fastest time for a champion?

In [82]:
fastest_winner = df_cleaned.sort_values('Time', ascending=True)
fastest_winner

#The fastest has been Mr. Margo Uusorg and Mrs. Birgit Ulrich with 55.5 seconds.

,Year,Carrier,Carried,Time,Country
3,2000,Margo Uusorg,Birgit Ulrich,55.5,Estonia
4,2001,Margo Uusorg,Birgit Ulrich,55.6,Estonia
9,2006,Margo Uusorg,Sandra Kullas,56.9,Estonia
8,2005,Margo Uusorg,Egle Soll,59.1,Estonia
6,2003,Margo Uusorg,Egle Soll,60.7,Estonia
14,2011,Taisto Miettinen,Kristiina Haapanen,60.7,Finland
15,2012,Taisto Miettinen,Kristiina Haapanen,61.2,Finland
10,2007,Madis Uusorg,Inga Klauso,61.7,Estonia
11,2008,Alar Voogla,Kirsti Viltrop,61.9,Estonia
12,2009,Taisto Miettinen,Kristiina Haapanen,62.0,Finland


Here in the end I download the data on my computer to use it in Datawrapper graphics.

In [84]:
file_path = 'C:/Users/Veera Tegelberg/OneDrive/Työpöytä/Lede_2024/Projects/project1/cleaned_data.csv'

df_cleaned.to_csv(file_path, index=False)

In [87]:

file_path = 'C:/Users/Veera Tegelberg/OneDrive/Työpöytä/Lede_2024/Projects/project1/most_wins.csv'
most_wins_per_pair.to_csv(file_path, index=False)
